In [8]:
import numpy as np
import pandas as pd
from matplotlib import patches
import matplotlib.pyplot as plt
from scipy.stats import *
from sklearn.preprocessing import normalize
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans

In [9]:
dataset='all_beta_center_p001_20190621_500PCs'

# phenotypes to analyze
#phe_codes=['INI21001','HC221','HC326', 'INI5255','HC276','INI5098','HC22','BIN_FC1002247','QT_FC1001717','INI1160','INI21002','INI20030690','INI20030620','INI20030700','INI40030700','INI1498','QT_FC1001180','BIN2207','INI20016','INI78','INI20030790','INI20030850','INI20022','INI20414','BIN_FC5006152','FH1263','BIN_FC1001747','BIN_FC1006149','FH1044','HC376','BIN_FC10002453','cancer1003','HC125','INI134','INI30000','INI2734','QT_FC1002375','cancer1002']
phe_codes = ['BIN_FC1002247']
#,'INI5255','HC276','HC326','HC382','INI3148','HC221']

# for regression models
covariate=['age','sex']+['PC'+str(i+1) for i in range(4)]

# load data
z=np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/all_pop/tsvd/'+
           dataset+'.npz', 
          allow_pickle = True)
scores=pd.read_csv('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/'+dataset+'.sscore',
                     index_col='#IID', sep = "\t")
phenos=pd.read_csv('/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.phe',
                     usecols=['IID']+phe_codes+covariate,
                     index_col='IID',
                     na_values=-9, sep = "\t")
npc=100

# analysis populations
val=set(pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/population-split/'+
                      'ukb24983_white_british_test.phe').iloc[:,0].tolist())

# phenotype names
with open('../../ukbb-tools/05_gbe/icdinfo.txt','r') as f:
    code_to_name = {line.split()[0]:line.split()[2].replace('_',' ').split('(')[0].capitalize() for line in f}

# program some special cases
#code_to_name['INI21001'] = 'Body mass index'
#code_to_name['HC294'] = 'Renal failure'
#code_to_name['HC326'] = 'Myocardial infarction'

/oak/stanford/groups/mrivas/anaconda/v3/envs/mrivas-env/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [ ]:
width = 2
plots = [None for _ in range(width * len(phe_codes))]
patterns = ('.', 'x', '-', '+', '\\')
plt.close('all'); plt.clf()
fig = plt.figure(figsize=(5*width + 1, 4*len(phe_codes)), dpi=300)
score_pcs=['SCORE{}_SUM'.format(pc+1) for pc in range(npc)]
profl_pcs=['PROF_PC{}'.format(pc+1) for pc in range(npc)]
for ix,phe_code in enumerate(phe_codes):
    for ix2 in range(width):
        plots[(width*ix)+ix2]=plt.subplot(len(phe_codes), width, width*ix + ix2 + 1)
    # scores and phenotypes
    phe_name=code_to_name[phe_code]
    df=phenos.loc[val,[phe_code]+covariate].dropna().merge(scores, left_index=True, right_index=True)
    weights=z['V'][np.where(z['label_phe_code'] == phe_code),:].flatten()[:npc]
    df['SCORE']=zscore(scores.loc[df.index.tolist(), score_pcs].dot(weights))
    df.sort_values(by='SCORE',ascending=False,inplace=True)
    
    # filter to cases for clustering binary traits
    if len(df[phe_code].value_counts()) == 2:
        is_bin=True
        # df=df[df[phe_code]==2]
    else:
        is_bin=False
    
    # do painting
    n=df.shape[0]
    for pc in range(npc):
        df[profl_pcs[pc]]=(df[score_pcs[pc]] * weights[pc] * df['SCORE']).clip_lower(0)
    df[profl_pcs] = normalize(df[profl_pcs], norm='l1')
    
    # identify high risk outliers
    df=df.iloc[:int((0.05 + (0.0*is_bin))*n),:]
    drp_centroid=df[profl_pcs].median()
    df['mahal']=df[profl_pcs].apply(lambda x: euclidean(x, drp_centroid), axis=1)
    m_star=df['mahal'].mean() + 2*df['mahal'].std()
    outliers=df.query('mahal > @m_star').index
    
    # perform clustering
    k=1
    cluster = KMeans(n_clusters=k, n_init=25).fit(df.loc[outliers,profl_pcs])
    pre_frac = 0.95 - 10.0/len(outliers)
    errors = [cluster.inertia_]
    for new_k in [2,3,4,5]:
        new_cluster = KMeans(n_clusters=new_k, n_init=25).fit(df.loc[outliers,profl_pcs])
        errors.append(new_cluster.inertia_)
        if new_k < 6 and (new_cluster.inertia_ / cluster.inertia_ < pre_frac**(new_k - k)): 
            cluster,k = new_cluster,new_k
    
    # (A): paint cluster centers    
    ms = cluster.cluster_centers_[0:k]
    c_ref = ['red','green','blue','black','magenta','yellow','cyan'][0:k]
    x,p = np.arange(k), []
    top5pcs = np.argsort(np.sum(ms, axis=0))[-5:]
    p=[]
    for i in range(ms.shape[1]):
        zz=plots[(width*ix)].bar(x, ms[:,i], bottom=np.sum(ms[:,0:i], axis=1), width=0.3,
                                 hatch=patterns[np.where(top5pcs==i)[0][0]] if i in top5pcs else None)
        p.append(zz)
    plots[(width*ix)].set_xlabel('')
    plots[(width*ix)].set_ylabel('Contribution to '+phe_name)
    plots[(width*ix)].set_ylim((0, 1))
    plots[(width*ix)].set_xlim((-0.4,k+0.4))
    plots[(width*ix)].set_xticks(np.arange(k))
    plots[(width*ix)].set_xticklabels([str(x+1)+'\n(n='+str(y)+')' for x,y in 
                              enumerate(list(pd.Series(cluster.labels_).value_counts()))])
    plots[(width*ix)].legend([p[i] for i in reversed(top5pcs)], 
                               ['PC' + str(i+1) for i in reversed(top5pcs)])
    
    # (B): scatterplot of clusters
    b,a=(str(i+1) for i in top5pcs[-2:])
    for ixx in range(k-1):
        plots[(width*ix)+1].plot([],[],'.',markersize=3,color=c_ref[ixx])
    plots[(width*ix)+1].scatter(df.loc[outliers,'SCORE'+a+'_SUM'].multiply(weights[int(a)-1]),
                                df.loc[outliers,'SCORE'+b+'_SUM'].multiply(weights[int(b)-1]),
                                c=[c_ref[i] for i in cluster.labels_])
    plots[(width*ix)+1].set_xlabel('PC'+a)
    plots[(width*ix)+1].set_ylabel('PC'+b)
    plots[(width*ix)+1].legend([patches.Circle((0.5, 0.5), 1, facecolor=c_ref[i], linewidth=3) for i in range(k)], 
                               ['Cluster '+str(i+1) for i in range(k)], loc='lower left')
    # add figure labels
    for ix2 in range(width):
        plots[(width*ix)+ix2].text(-.1, 1.05, chr((ix%len(phe_codes))+65+ix2*len(phe_codes)), 
                                   fontsize=20, transform=plots[(width*ix)+ix2].transAxes)
    

plots[-2].set_xlabel('Cluster')
plt.show()
fig.savefig("fig4hearing.pdf", bbox_inches='tight')